In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split

import numpy as np
import pandas as pd

from utils.path_utils import Files
from utils.config_base import print_initialization
from neural.config import *
from evaluation.analysis import Plotter

from neural.dataset import numpyDataset
from neural.mlp import MLP, MlpBlock, RMLP
from neural.training import Trainer, MLPTrainer
from neural.train_results import FitResult



pd.options.display.max_columns = 30

torch.set_printoptions(sci_mode=False)

In [ ]:
class_name = "MLP"
params = {'in_dim': 2,
 'dims': [3],
 'nonlins': ['none'],
 }
MLP.__init__.__annotations__
globals()[class_name](**params)

In [ ]:
MlpBlock(2, [3,4,5], ['relu']*3)

In [ ]:
log_path = './data/Exp1_GuyGilad_logs_yolo/bboxes2.csv'

io_config = IOConfig(
    input_frames = [0,-3,-15,-18,-30], # list
    pred_frames = [12], # list
)

dataset_config = DatasetConfig.from_ioConfig(io_config, log_path)


In [ ]:
# Current Best Model
hidden_dims = [120, 40, 12, 4, 2]
activaations = ['relu'] * (len(hidden_dims))# + ['softmax']
in_dim = 4 * len(dataset_config.input_frames)
model = MLP(in_dim=in_dim, dims=hidden_dims, nonlins=activaations)


In [ ]:
block_in_dim = 40
block_dims = [10, 4, 10, 40]
block_activations = ["relu"] * (len(block_dims))
in_dim = 4 * len(dataset_config.input_frames)
out_dim = 2 * len(dataset_config.pred_frames)

model = RMLP(
    in_dim=in_dim,
    block_in_dim=block_in_dim,
    block_dims=block_dims,
    block_nonlins=block_activations,
    n_blocks=4,
    out_dim=out_dim,
)

In [ ]:
print(model)

In [ ]:
train_config = TrainConfig(
    seed = 42, # int
    dataset = dataset_config, # Dataset
    model = model, # Union[nn.Module, str]
    loss_fn = nn.MSELoss, # nn.Module
    optimizer = 'adam', # Union[Optimizer, str]
    device = f"cpu", # str
    log = True, # bool
    num_epochs = 100, # int
    checkpoints = "RMLP4", # str
    early_stopping = 15, # int
    print_every = 5, # int
    learning_rate = 0.001, # float
    weight_decay = 1e-05, # float
    batch_size = 256, # int
    shuffle = True, # bool
    num_workers = 0, # int
    train_test_split = 0.8, # float
)

In [ ]:
dataset = numpyDataset.create_from_config(dataset_config, save_path=None)

In [ ]:
dataset[:10]

In [ ]:
# Split the dataset
ds_train, ds_test = random_split(dataset, [train_config.train_test_split, 1- train_config.train_test_split])

In [ ]:
# Create the dataloaders
dl_train = DataLoader(ds_train, batch_size=train_config.batch_size, shuffle=train_config.shuffle)
dl_test = DataLoader(ds_test, batch_size=train_config.batch_size, shuffle=train_config.shuffle)

In [ ]:
loss_fn = train_config.loss_fn()
lr = train_config.learning_rate
weight_decay = train_config.weight_decay
optimizer = train_config.optimizer(model.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
trainer = MLPTrainer(model, loss_fn, optimizer, device=train_config.device, log=train_config.log)

In [ ]:
epochs = train_config.num_epochs
checkpoints = train_config.checkpoints
early_stopping = train_config.early_stopping
print_every = train_config.print_every

trainer.fit(
    dl_train=dl_train,
    dl_test=dl_test,
    num_epochs=epochs,
    checkpoints=checkpoints,
    print_every=print_every,
    early_stopping=early_stopping,
)

In [ ]:
train_config.save_pickle(trainer.logger.log_dir + "/train_config.pkl")